### Imports

In [1]:
import igraph as ig
from random import uniform

# visualisation
import seaborn as sns
import matplotlib.pyplot as plt

# SPEED
#from multiprocessing import Pool

ImportError: Unable to import required dependencies:
pytz: No module named 'pytz'

### Model Definition

In [2]:
# Nodes are required to have the following values:
# name
# decision - takes a value from [-1, 0, 1] (decline, undecided, buy))
# qexpectation - float between 0 and 1


class Model:
    # for lattice:
    def __init__(self, n: int):
        self.n = n
        self.g = self.get_lattice()

    def decision(self, n, qexp, Q):
        return qexp - (n * 0.05) < Q

    def get_lattice(self):
        lattice = ig.Graph()
        for i in range(self.n):
            for j in range(self.n):
                lattice.add_vertex(name=f"{i},{j}"
                                   , decision=0
                                   , qexpectation=uniform(0, 1))
        for i in range(self.n-1):
            for j in range(self.n-1):
                lattice.add_edge(f"{i},{j}", f"{i+1},{j}")
                lattice.add_edge(f"{i},{j}", f"{i},{j+1}")
        for i in range(self.n-1):
            lattice.add_edge(f"{i},{self.n - 1}", f"{i + 1},{self.n - 1}")
            lattice.add_edge(f"{self.n - 1},{i}", f"{self.n - 1},{i + 1}")
        return lattice

    def lattice_to_array (self):
        array = []
        for i in range(self.n):
            row = []
            for j in range(self.n):
                row.append(self.g.vs.find(f'{i},{j}')['decision'])
            array.append(row)
        return array

    def print_lattice(self):
        for i in range(self.n):
            for j in range(self.n):
                print(self.g.vs.find(f'{i},{j}')['decision'], end=' ')
            print('')

    def steps(self, prev_n: [ig.Vertex], q: float):
        if not prev_n:
            pass
        else:
            next_n = set()
            for n in prev_n:
                neighbours = n.neighbors()
                if self.decision(len([x for x in neighbours if x['decision'] == 1]), n['qexpectation'], q):
                    n['decision'] = 1
                    next_n.update(set(neighbours))
                else:
                    n['decision'] = -1   
            next_n = list((filter(lambda x: x['decision'] == 0, next_n)))
            self.steps(next_n, q)




In [3]:
n = 50
Qs = [.55, .57, .59, .61, .63]

for Q in Qs:
    network = Model(n)
    sx = n-1
    sy = int(n/2)
    network.g.vs.find(f'{sx},{int(sy)}')['decision'] = 1
    network.steps(network.g.vs.find(f'{sx},{sy}').neighbors(), Q)
    fig = plt.figure()
    x = sns.heatmap(network.lattice_to_array())

NameError: name 'plt' is not defined

### TODO:
1. We need to be able to schedule multiple waves of product. Modify `steps` so that next_ns are 0's and 1's - this way we have a chance of futher propagating with multiple waves (currently will just die on first iteration of new `steps` execution). Idea: accumulate chance of buying over supersteps by **reducing** quality expectation for each neighbour for each wave.As per:
```
(b) (...) and then these bond probabilities increase with time if neighboring sites havebought a product. Then, if one allows for the existence of successive product waves(annual issues of the car models, movies in a series like Star Wars, Pink Panther,etc.) 
```
and
```
(g) On top of this one can consider modeling the e ects of peer pressure: siteswhich are not potential buyers becoming buyers when many of the neighbors boughtthe product. Sometimes this is not just a psychological e ect: it is related with theutility of the product depending on its use by the other buyers (like in the case of fax,ps, pdf, word  les formats)
```
1. Just take one of the graphs and try replicating it. I got quite nice successes with the percolation itself
1. If we got any of the plots above run them on Watts-Strogatz or something